<a href="https://colab.research.google.com/github/ArunK-ML/Project---Shopper-Spectrum-Customer-Segmentation-and-Product-Recommendations-in-E-Commerce/blob/main/Shopper_Spectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🛒 Shopper Spectrum: Customer Segmentation and Product Recommendations in E-Commerce**

# **Step 1:Dataset Collection and understanding**

In [1]:
import pandas as pd
df = pd.read_csv('/content/online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2022-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2022-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
df.shape

(398517, 8)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398517 entries, 0 to 398516
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    398517 non-null  object 
 1   StockCode    398517 non-null  object 
 2   Description  397187 non-null  object 
 3   Quantity     398517 non-null  int64  
 4   InvoiceDate  398517 non-null  object 
 5   UnitPrice    398517 non-null  float64
 6   CustomerID   296110 non-null  float64
 7   Country      398516 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 24.3+ MB


In [8]:
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [5]:
df.duplicated().sum()

np.int64(3202)

# **Step 2: 📌 Data Preprocessing:**

**Handle Null Value**

In [7]:
df = df.dropna(subset=["CustomerID","Country"])

df["Description"] = df["Description"].fillna("No Description")

**Exclude cancelled invoices (InvoiceNo starting with 'C')**

In [9]:
# Remove cancelled invoices (InvoiceNo starting with 'C')
df = df[~df["InvoiceNo"].astype(str).str.startswith("C")]

**Remove negative or zero quantities and prices**

In [10]:
# Keep only rows where Quantity > 0 and UnitPrice > 0
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

**DateTime DataType Conversation**

In [13]:
# Parse invoice date
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Split into Date and Time
df["InvoiceDay"] = df["InvoiceDate"].dt.date
df["InvoiceTime"] = df["InvoiceDate"].dt.time


# **Step 3 :📌 Exploratory Data Analysis (EDA):**

In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,InvoiceTime
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2022-12-01 08:26:00,2.55,17850.0,United Kingdom,2022-12-01,08:26:00
1,536365,71053,WHITE METAL LANTERN,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom,2022-12-01,08:26:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2022-12-01 08:26:00,2.75,17850.0,United Kingdom,2022-12-01,08:26:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom,2022-12-01,08:26:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2022-12-01 08:26:00,3.39,17850.0,United Kingdom,2022-12-01,08:26:00


In [15]:
df.shape

(288997, 10)

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from datetime import timedelta

# Add revenue column
# df["Revenue"] = df["Quantity"] * df["UnitPrice"]

In [ ]:
# -----------------------------
# 1) Transaction volume by country
# -----------------------------
tx_by_country = (
    df.groupby("Country")["InvoiceNo"]
      .nunique()
      .sort_values(ascending=False)
)
print("\n📌 Transaction volume by country (unique invoices):")
print(tx_by_country.head(10))

tx_by_country.head(10).plot(kind="barh", title="Top 10 Countries by Invoices")
plt.show()

In [ ]:
# -----------------------------
# 2) Top-selling products
# -----------------------------
# By quantity
prod_qty = (
    df.groupby(["StockCode", "Description"])["Quantity"]
      .sum()
      .sort_values(ascending=False)
)
print("\n📌 Top 10 Products by Quantity Sold:")
print(prod_qty.head(10))

# By revenue
prod_rev = (
    df.groupby(["StockCode", "Description"])["Revenue"]
      .sum()
      .sort_values(ascending=False)
)
print("\n📌 Top 10 Products by Revenue:")
print(prod_rev.head(10))

In [ ]:
# -----------------------------
# 3) Purchase trends over time
# -----------------------------
daily = df.set_index("InvoiceDate").resample("D").agg(
    Revenue=("Revenue", "sum"),
    Invoices=("InvoiceNo", "nunique")
)

daily["Revenue"].plot(title="Daily Revenue Over Time")
plt.show()

daily["Invoices"].plot(title="Daily Invoice Counts Over Time")
plt.show()

In [ ]:
# -----------------------------
# 4) Monetary distribution
# -----------------------------
invoice_totals = df.groupby("InvoiceNo")["Revenue"].sum()
customer_totals = df.groupby("CustomerID")["Revenue"].sum()

plt.hist(invoice_totals, bins=50)
plt.title("Distribution of Invoice Totals")
plt.show()

plt.hist(customer_totals, bins=50)
plt.title("Distribution of Customer Totals")
plt.show()


In [ ]:
# -----------------------------
# 5) RFM analysis
# -----------------------------
ref_date = df["InvoiceDate"].max() + timedelta(days=1)

rfm = df.groupby("CustomerID").agg(
    Recency=("InvoiceDate", lambda x: (ref_date - x.max()).days),
    Frequency=("InvoiceNo", "nunique"),
    Monetary=("Revenue", "sum")
).reset_index()

print("\n📌 RFM Sample:")
print(rfm.head())

# Histograms
rfm[["Recency"]].hist(bins=50)
plt.title("Recency Distribution")
plt.show()

rfm[["Frequency"]].hist(bins=50)
plt.title("Frequency Distribution")
plt.show()

rfm[["Monetary"]].hist(bins=50)
plt.title("Monetary Distribution")
plt.show()



In [ ]:
# -----------------------------
# 6) Elbow curve for KMeans clustering
# -----------------------------
X = rfm[["Recency", "Frequency", "Monetary"]]
X_scaled = StandardScaler().fit_transform(X)

inertias = []
for k in range(1, 11):
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.plot(range(1, 11), inertias, marker="o")
plt.title("Elbow Curve (RFM Clustering)")
plt.xlabel("Clusters (k)")
plt.ylabel("Inertia")
plt.show()

# Choose k=4 as example
km_final = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm["Cluster"] = km_final.fit_predict(X_scaled)

print("\n📌 Customer Cluster Profiles:")
print(rfm.groupby("Cluster").mean())

In [ ]:
# -----------------------------
# 7) Product recommendation heatmap
# -----------------------------
# Pick top N products
topN = 30
top_prod_codes = (
    df.groupby("StockCode")["InvoiceNo"]
      .nunique()
      .sort_values(ascending=False)
      .head(topN)
      .index
)

df_top = df[df["StockCode"].isin(top_prod_codes)]
cust_prod = (
    df_top.assign(Buy=1)
          .pivot_table(index="CustomerID", columns="StockCode", values="Buy", aggfunc="max", fill_value=0)
)

sim_matrix = pd.DataFrame(
    cosine_similarity(cust_prod.T),
    index=cust_prod.columns,
    columns=cust_prod.columns
)

plt.imshow(sim_matrix, cmap="viridis")
plt.xticks(range(len(sim_matrix.columns)), sim_matrix.columns, rotation=90)
plt.yticks(range(len(sim_matrix.index)), sim_matrix.index)
plt.title("Product Similarity Heatmap (Top 30 Products)")
plt.colorbar()
plt.show()
